# ETH 15m — Goldbach Strategy

This notebook fetches 15-minute ETH/USD candles from Coinbase Pro, computes indicators, detects pivots, labels Goldbach phases, and runs a simple backtest using the PDF rules.

# --- Algo classifier: which algo and subcase are we in?
def which_algo(row, df=None):
    # Uses EMA, RSI, BB width and pivot proximity to choose:
    # Algo 1 = Trend-following (look for continuation entries)
    # Algo 2 = Mean-reversion / Pullback (look for fade entries)
    ema_fast = row.get('ema20', row.get('ema13', None))
    ema_slow = row.get('ema50', None)
    rsi = row.get('rsi14', None)
    bb_w = row.get('bb_w', None)
    close = row['close']
    atr = row.get('atr14', None)

    # pivot proximity (optional)
    prox = 'far'
    if df is not None and atr is not None and not pd.isna(atr):
        idx_low, lvl_low = recent_pivot_level(df, 'low', lookback=200)
        idx_high, lvl_high = recent_pivot_level(df, 'high', lookback=200)
        if lvl_low is not None and abs(close - lvl_low) <= 1.0 * atr:
            prox = 'near_low'
        if lvl_high is not None and abs(close - lvl_high) <= 1.0 * atr:
            prox = 'near_high'

    # Momentum thresholds
    strong_mom = (rsi is not None and rsi > 60)
    weak_mom = (rsi is not None and rsi < 40)

    # Volatility low means range (favours mean reversion), high favours continuation breakouts
    low_vol = (bb_w is not None and bb_w < BB_WIDTH_LOW)
    high_vol = (bb_w is not None and bb_w > BB_WIDTH_HIGH)

    # Trend
    if ema_fast is not None and ema_slow is not None:
        is_up = ema_fast > ema_slow
        is_down = ema_fast < ema_slow
    else:
        is_up = is_down = False

    # Heuristic classifier:
    # Algo 1 (trend-following) if clear trend and momentum or high volatility
    if (is_up and strong_mom) or (is_down and weak_mom) or high_vol:
        algo = 'Algo 1 (trend-following)'
        if is_up:
            sub = 'Bull continuation — look for breakouts or trend pullbacks'
        elif is_down:
            sub = 'Bear continuation — look for short breakouts/pullbacks'
        else:
            sub = 'Volatility breakout regime'

    # Algo 2 (mean-reversion) if range or near pivot with neutral momentum
    elif low_vol or prox.startswith('near') or (not strong_mom and not weak_mom):
        algo = 'Algo 2 (mean-reversion/pullback)'
        if prox == 'near_low':
            sub = 'Long fade near pivot low (support) — look for tight stops'
        elif prox == 'near_high':
            sub = 'Short fade near pivot high (resistance) — look for tight stops'
        else:
            sub = 'Range trades / mean reversion'
    else:
        algo = 'Unknown / mixed'
        sub = 'Transition; use smaller size or wait for confirmation'

    return {'algo': algo, 'subcase': sub, 'prox': prox, 'rsi': rsi, 'ema_fast': ema_fast, 'ema_slow': ema_slow, 'bb_w': bb_w}
# run classifier on last bar
cls = which_algo(cur, df=df)
print('Algo classification for last bar:')
import json
print(json.dumps(cls, indent=2, default=str))
Quick steps:
- Ensure dependencies are installed in your kernel: `%pip install -r requirements.txt` or run the PowerShell command shown in the README.
- Run cells from top to bottom. The main user-facing cells are: 1) Parameters, 2) Fetch & indicators, 3) Pivot & phase logic, 4) PDF rule integration, 5) Backtester & results.

What this cleaned notebook includes:
- Clear parameters cell (Coinbase, pair, timeframe) you can edit.
- Clean, documented functions for pivots, phase labeling, and divergence detection.
- A simple, illustrative backtester using pivot-based stops and slippage/fee modeling.

Notes:
- I preserved the Goldbach logic from the PDF but used reasonable defaults where the PDF didn't provide exact numbers. Tell me which thresholds to change and I'll update them.

In [ ]:
# Improved pivot detector and refined Goldbach phase mapping
import math
import ta
import pandas as pd


def find_pivots_df(df, col='close', left=5, right=5, min_atr_mult=0.5):
    """Detect pivots using a left/right window and require minimum prominence relative to ATR.

    Returns a dict mapping timestamp -> 'high' or 'low'.
    """
    pivots = {}
    prices = df[col].values
    # compute ATR for prominence threshold (use 14-period ATR)
    atr_obj = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    atr = atr_obj.average_true_range().fillna(method='backfill').values
code
python
# Classifier confidence: compute z-blend score for last bar and print classifier + confidence
row = df.iloc[-1]
# z-scores (use dataset stats)
rsi_z = (row['rsi14'] - df['rsi14'].mean()) / (df['rsi14'].std() or 1.0)
bbw_z = (row['bb_w'] - df['bb_w'].mean()) / (df['bb_w'].std() or 1.0)
ema_z = ((row['ema20'] - row['ema50']) / row['ema50'] - ((df['ema20'] - df['ema50'])/df['ema50']).mean()) / (((df['ema20'] - df['ema50'])/df['ema50']).std() or 1.0)
conf_score = %d * ema_z + %d * rsi_z + %d * bbw_z
conf = 1.0 / (1.0 + np.exp(-conf_score))
cls = which_algo(row, df=df)
print('Classifier:')
import json
print(json.dumps(cls, indent=2))
print(f'Confidence (0-1): {conf:.3f}, raw_score: {conf_score:.3f}')

# Quick: run long-only backtest via the script (will call its backtest with LONG_ONLY True if wired)
# In this notebook we can import the run_backtest module and call backtest_goldbach if available.
try:
    import run_backtest
    print('Imported run_backtest; running long-only backtest (this may take a few moments)')
    run_backtest.LONG_ONLY = True
    run_backtest.ENABLE_SHORTS = False
    trades_df, metrics = run_backtest.backtest_goldbach(df)
    print('Long-only backtest done. Metrics:')
    print(metrics)
except Exception as e:
    print('Could not run long-only backtest here:', e)

    n = len(prices)
    for i in range(left, n - right):
        window = prices[i - left:i + right + 1]
        v = prices[i]
        local_mean = window.mean()
        prom = abs(v - local_mean)
        # require local extremum and minimum prominence relative to ATR at i
        if v == window.max() and prom >= max(0.0, min_atr_mult * atr[i]):
            pivots[df.index[i]] = 'high'
        elif v == window.min() and prom >= max(0.0, min_atr_mult * atr[i]):
            pivots[df.index[i]] = 'low'
    return pivots


def recent_pivot_level(df, kind='low', lookback=200):
    """Return (timestamp, price) of the most recent pivot of `kind` within lookback bars or (None, None)."""
    recent = df.tail(lookback)
    if 'pivot' not in recent.columns:
        return None, None
    mask = recent['pivot'] == kind
    if mask.any():
        idx = recent[mask].index[-1]
        return idx, recent.at[idx, 'close']
    return None, None


def goldbach_phase_refined(row, df=None):
    """Refined phase labeling: trend (EMA), momentum (RSI), volatility (BB width), and pivot proximity."""
    ema20 = row['ema20']
    ema50 = row['ema50']
    rsi = row['rsi14']
    bb_w = row['bb_w']
    close = row['close']
    atr = row.get('atr14', float('nan'))

    # Trend
    if ema20 > ema50:
        trend = 'bull'
    elif ema20 < ema50:
        trend = 'bear'
    else:
        trend = 'flat'

    # Volatility
    if pd.isna(bb_w):
        vol = 'unknown'
    elif bb_w > BB_WIDTH_HIGH:
        vol = 'high'
    elif bb_w < BB_WIDTH_LOW:
        vol = 'low'
    else:
        vol = 'normal'

    # Momentum
    if pd.isna(rsi):
        mom = 'unknown'
    elif rsi > RSI_OB:
        mom = 'overbought'
    elif rsi < RSI_OS:
        mom = 'oversold'
    else:
        mom = 'neutral'

    # Proximity to recent pivots (if df provided)
    prox = 'far'
    if df is not None and not pd.isna(atr) and atr > 0:
        idx_low, lvl_low = recent_pivot_level(df, 'low', lookback=200)
        idx_high, lvl_high = recent_pivot_level(df, 'high', lookback=200)
        if lvl_low is not None and abs(close - lvl_low) <= 1.0 * atr:
            prox = 'near_low'
        if lvl_high is not None and abs(close - lvl_high) <= 1.0 * atr:
            prox = 'near_high'

    # Phase rules
    if vol == 'low':
        phase = 'Ranging/Consolidation'
    elif trend == 'bull' and mom in ('neutral', 'oversold'):
        phase = 'Bull Pullback (support test)' if prox == 'near_low' else 'Bull Trend'
    elif trend == 'bear' and mom in ('neutral', 'overbought'):
        phase = 'Bear Pullback (resistance test)' if prox == 'near_high' else 'Bear Trend'
    elif mom == 'oversold' and vol == 'high':
        phase = 'Reversal Candidate (bear exhausted)'
    elif mom == 'overbought' and vol == 'high':
        phase = 'Reversal Candidate (bull exhausted)'
    else:
        phase = 'Unknown/Transition'

    # Score: trend bias plus momentum/volatility modifier
    score = 0
    score += 1 if trend == 'bull' else -1 if trend == 'bear' else 0
    if mom == 'overbought':
        score -= 1
    if mom == 'oversold':
        score += 1
    if vol == 'high':
        score *= 2

    return phase, trend, mom, vol, score


In [ ]:
# Integrate PDF-derived rules: RSI divergence detection and PDF-driven adjustments

def detect_rsi_divergence(df, kind='high'):
    """Detect simple regular divergence using the last two pivots of specified kind.
    Returns (bool, info) where info contains pivot timestamps.
    """
    if 'pivot' not in df.columns:
        return False, None
    piv = df.dropna(subset=['pivot'])[['close', 'rsi14', 'pivot']]
    piv = piv[piv['pivot'] == kind]
    if len(piv) < 2:
        return False, None
    last_two = piv.tail(2)
    p1 = last_two.iloc[0]
    p2 = last_two.iloc[1]
    price1, price2 = p1['close'], p2['close']
    rsi1, rsi2 = p1['rsi14'], p2['rsi14']
    if kind == 'high':
        # bearish regular divergence: price makes higher high, RSI makes lower high
        if price2 > price1 and rsi2 < rsi1:
            return True, {'type': 'bear_reg', 'pivots': list(last_two.index)}
    else:
        # bullish regular divergence: price makes lower low, RSI makes higher low
        if price2 < price1 and rsi2 > rsi1:
            return True, {'type': 'bull_reg', 'pivots': list(last_two.index)}
    return False, None


def goldbach_phase_with_pdf(row, df=None):
    """Wrap goldbach_phase_refined and apply PDF-specific logic:
    - Detect divergences and downgrade/flag phases accordingly
    - Provide entry/stop suggestions based on recent pivots

    Returns: phase, trend, mom, vol, score, meta_dict
    """
    phase, trend, mom, vol, score = goldbach_phase_refined(row, df=df)
    div_bear, info_b = (False, None)
    div_bull, info_l = (False, None)
    if df is not None:
        div_bear, info_b = detect_rsi_divergence(df, kind='high')
        div_bull, info_l = detect_rsi_divergence(df, kind='low')
    # Apply PDF guidance
    if div_bear and trend == 'bull':
        phase = 'Reversal Candidate (bear divergence)'
        score -= 2
    if div_bull and trend == 'bear':
        phase = 'Reversal Candidate (bull divergence)'
        score += 2

    # Entry/stop suggestions
    entry_suggestion = None
    stop_suggestion = None
    if trend == 'bull' and df is not None:
        idx_low, lvl_low = recent_pivot_level(df, 'low', lookback=200)
        if lvl_low is not None and row.get('atr14') is not None:
            entry_suggestion = f'Pullback to EMA{EMA_FAST} or pivot low ~{lvl_low:.2f}'
            stop_suggestion = max(0.0, lvl_low - 0.5 * row['atr14'])
        else:
            entry_suggestion = f'Pullback to EMA{EMA_FAST} or mid-BB'
    if trend == 'bear' and df is not None:
        idx_high, lvl_high = recent_pivot_level(df, 'high', lookback=200)
        if lvl_high is not None and row.get('atr14') is not None:
            entry_suggestion = f'Pullback to EMA{EMA_FAST} or pivot high ~{lvl_high:.2f}'
            stop_suggestion = lvl_high + 0.5 * row['atr14']
        else:
            entry_suggestion = f'Pullback to EMA{EMA_FAST} or mid-BB'

    meta = {'div_bear': info_b, 'div_bull': info_l, 'entry': entry_suggestion, 'stop': stop_suggestion}
    return phase, trend, mom, vol, score, meta


In [ ]:
# Override parameters: use Coinbase and USD pair
EXCHANGE_ID = 'coinbase'  # CCXT id for Coinbase (public market data)
SYMBOL = 'ETH/USD'         # Coinbase uses USD rather than USDT
TIMEFRAME = '15m'
LIMIT = 800  # increase lookback for backtest stability

print(f'Parameters overridden: {EXCHANGE_ID}, {SYMBOL}, timeframe={TIMEFRAME}, limit={LIMIT}')

# Updates: Coinbase Pro + tightened pivots + simple backtester

This notebook was extended to:

- Use Coinbase Pro (ETH/USD) instead of Binance.
- Include a stricter pivot detector (ATR-based prominence) and improved Goldbach phase mapping.
- Add a small, illustrative backtester for the simple Goldbach rules.

Run the notebook from top to bottom to ensure the new cells override prior definitions.

# ETH 15m — Goldbach-inspired analysis
This notebook fetches 15-minute ETH/USDT candles from Binance (via CCXT), computes simple indicators (EMA20/50, RSI14, Bollinger Bands), detects pivots, and labels each bar with a simplified Goldbach phase.

Assumptions (tunable): EMA20/EMA50 trend, RSI(14) momentum (70/30), Bollinger(20,2) width thresholds, pivot lookback=3.

Use the cell below to install requirements in the active kernel if needed: `%pip install -r requirements.txt` or run the equivalent in PowerShell.

In [ ]:
# Optional: install (run in notebook kernel)
# %pip install ccxt pandas numpy ta python-dateutil

import ccxt
import pandas as pd
import numpy as np
import ta
from datetime import datetime, timezone
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.6f}'.format

In [ ]:
# Parameters (tweak these to match your Goldbach rules)
EXCHANGE_ID = 'coinbase'   # use Coinbase public market data
SYMBOL = 'ETH/USD'            # Coinbase uses USD (not USDT)
TIMEFRAME = '15m'
LIMIT = 1200                  # number of candles to fetch (increase for backtests)

# Indicator settings (tuned)
EMA_FAST = 13    # faster short EMA for more responsive pullback detection
EMA_SLOW = 55
RSI_LEN = 14
RSI_OB = 72
RSI_OS = 28
BB_WINDOW = 20
BB_DEV = 2
BB_WIDTH_LOW = 0.015   # narrower low-vol threshold
BB_WIDTH_HIGH = 0.07   # larger high-vol threshold

# Pivot / sizing / risk (tuned)
PIVOT_LOOKBACK = 5
STOP_ATR_MULT = 1.8

# Backtest / execution defaults
BACKTEST_RISK_PCT = 0.01   # risk 1% of capital per trade
BACKTEST_SLIPPAGE = 0.0008 # 0.08% slippage assumption
BACKTEST_FEE = 0.0015      # 0.15% per trade fee assumption

print(f'Parameters set: {EXCHANGE_ID} {SYMBOL} {TIMEFRAME} limit={LIMIT}')

In [ ]:
# Fetch OHLCV from exchange and compute indicators
print('Fetching OHLCV...')
exchange = getattr(ccxt, EXCHANGE_ID)()
raw = exchange.fetch_ohlcv(SYMBOL, timeframe=TIMEFRAME, limit=LIMIT)
df = pd.DataFrame(raw, columns=['timestamp','open','high','low','close','volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
df.set_index('timestamp', inplace=True)

# Basic indicators
df['ema20'] = df['close'].ewm(span=EMA_FAST, adjust=False).mean()
df['ema50'] = df['close'].ewm(span=EMA_SLOW, adjust=False).mean()
df['ma10'] = df['close'].rolling(10).mean()
df['rsi14'] = ta.momentum.rsi(df['close'], window=RSI_LEN)
bb = ta.volatility.BollingerBands(df['close'], window=BB_WINDOW, window_dev=BB_DEV)
df['bb_h'] = bb.bollinger_hband()
df['bb_l'] = bb.bollinger_lband()
df['bb_m'] = bb.bollinger_mavg()
df['bb_w'] = (df['bb_h'] - df['bb_l']) / df['bb_m']
df['bb_pos'] = (df['close'] - df['bb_l']) / (df['bb_h'] - df['bb_l'])
atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14)
df['atr14'] = atr.average_true_range()

# Detect pivots and write to df
pivots = find_pivots_df(df, col='close', left=PIVOT_LOOKBACK, right=PIVOT_LOOKBACK, min_atr_mult=0.5)
df['pivot'] = pd.NA
for idx, kind in pivots.items():
    if idx in df.index:
        df.at[idx, 'pivot'] = kind

print('OHLCV fetched and indicators computed. Rows:', len(df))

In [ ]:
# Backtester updated to use Goldbach PDF rules with long + short logic and CSV export
import math
import os

def backtest_goldbach(df, initial_capital=10000.0, risk_pct=BACKTEST_RISK_PCT, slippage=BACKTEST_SLIPPAGE, fee=BACKTEST_FEE):
    df = df.copy().dropna(subset=['ema20','ema50','atr14'])
    capital = initial_capital
    position = 0.0  # units of ETH
    side = None
    entry = None
    trades = []

    for i in range(50, len(df)-1):
        row = df.iloc[i]
        next_open = df.iloc[i+1]['open']
        phase, trend, mom, vol, score, meta = goldbach_phase_with_pdf(row, df=df.iloc[:i+1])
        div_bear = meta.get('div_bear')
        div_bull = meta.get('div_bull')

        # Entry: LONG
        if position == 0:
            if trend == 'bull' and (row['close'] <= row['ema20'] * 1.002 or (meta.get('entry') and 'pivot low' in str(meta.get('entry')))) and row['rsi14'] > 40 and not div_bear:
                idx_low, lvl_low = recent_pivot_level(df.iloc[:i+1], 'low', lookback=200)
                stop = (lvl_low - 0.5 * row['atr14']) if lvl_low is not None else row['close'] - STOP_ATR_MULT * row['atr14']
                risk_per_unit = next_open - stop if next_open > stop else row['atr14']
                if risk_per_unit <= 0:
                    continue
                usd_risk = capital * risk_pct
                units = usd_risk / risk_per_unit
                entry_price = next_open * (1 + slippage)
                position = units
                side = 'long'
                entry = {'side':'long','entry':entry_price,'stop':stop,'units':units,'entry_idx':row.name,'fee':fee,'slippage':slippage}
                trades.append(entry.copy())

            # Entry: SHORT
            elif trend == 'bear' and (row['close'] >= row['ema20'] * 0.998 or (meta.get('entry') and 'pivot high' in str(meta.get('entry')))) and row['rsi14'] < 60 and not div_bull:
                idx_high, lvl_high = recent_pivot_level(df.iloc[:i+1], 'high', lookback=200)
                stop = (lvl_high + 0.5 * row['atr14']) if lvl_high is not None else row['close'] + STOP_ATR_MULT * row['atr14']
                risk_per_unit = stop - next_open if stop > next_open else row['atr14']
                if risk_per_unit <= 0:
                    continue
                usd_risk = capital * risk_pct
                units = usd_risk / risk_per_unit
                entry_price = next_open * (1 - slippage)
                position = -units
                side = 'short'
                entry = {'side':'short','entry':entry_price,'stop':stop,'units':units,'entry_idx':row.name,'fee':fee,'slippage':slippage}
                trades.append(entry.copy())

        else:
            # Manage open position: check stop or target 2x
            current_price = row['close']
            last = trades[-1]
            stop = last['stop']
            if last['side'] == 'long':
                target = last['entry'] + 2*(last['entry'] - stop)
                if current_price <= stop:
                    exit_price = current_price * (1 - slippage) - fee*current_price
                    pnl = (exit_price - last['entry']) * last['units']
                    capital += pnl
                    last.update({'exit':exit_price,'pnl':pnl,'exit_idx':row.name})
                    position = 0
                    side = None
                elif current_price >= target:
                    exit_price = target * (1 - slippage) - fee*target
                    pnl = (exit_price - last['entry']) * last['units']
                    capital += pnl
                    last.update({'exit':exit_price,'pnl':pnl,'exit_idx':row.name})
                    position = 0
                    side = None
            else:  # short
                target = last['entry'] - 2*(stop - last['entry'])
                if current_price >= stop:
                    exit_price = current_price * (1 + slippage) + fee*current_price
                    pnl = (last['entry'] - exit_price) * last['units']
                    capital += pnl
                    last.update({'exit':exit_price,'pnl':pnl,'exit_idx':row.name})
                    position = 0
                    side = None
                elif current_price <= target:
                    exit_price = target * (1 + slippage) + fee*target
                    pnl = (last['entry'] - exit_price) * last['units']
                    capital += pnl
                    last.update({'exit':exit_price,'pnl':pnl,'exit_idx':row.name})
                    position = 0
                    side = None

    # Summarize and export trades
    trades_df = pd.DataFrame(trades)
    if not trades_df.empty:
        trades_df['pnl'] = trades_df.get('pnl', pd.Series([None]*len(trades_df)))
    wins = trades_df[trades_df['pnl']>0] if not trades_df.empty else pd.DataFrame()
    losses = trades_df[trades_df['pnl']<=0] if not trades_df.empty else pd.DataFrame()
    total_pnl = trades_df['pnl'].sum() if not trades_df.empty else 0.0
    print('Backtest summary (Goldbach rules integrated)')
    print(f'Trades: {len(trades_df)}, Wins: {len(wins)}, Losses: {len(losses)}, PnL: {total_pnl:.2f}, Final capital: {capital:.2f}')

    # Save CSV
    out_path = os.path.join(os.getcwd(), 'goldbach_trades.csv')
    trades_df.to_csv(out_path, index=False)
    print(f'Trades exported to: {out_path}')
    return trades_df

# Run the updated backtester and save
trades_df = backtest_goldbach(df.copy())
# show first 5 trades
trades_df.head(5)


In [ ]:
# Current snapshot: last candle, phase, and suggested entry/stop
cur = df.iloc[-1]
phase, trend, mom, vol, score, meta = goldbach_phase_with_pdf(cur, df=df)
snapshot = {
    'time_utc': df.index[-1],
    'price': cur['close'],
    'phase': phase,
    'trend': trend,
    'momentum': mom,
    'volatility': vol,
    'score': score,
    'entry_suggestion': meta.get('entry'),
    'stop_suggestion': meta.get('stop')
}
import pandas as _pd
print('Current snapshot:')
print(_pd.DataFrame([snapshot]).T)
